In [ ]:
import tweepy
import csv
import time

# Twitter API credentials
api_key = "ufmmiP6WPrf88KFWBucUYqyqj"
api_secret = "VC5lt1Wa6Y4TDsMByVSzRe5iB36XXNMdFrD99yibpTKt4rHtXS"
access_token = "1483027022645706754-4JCnTuUQ8FfUMBS8MOre8cufJBK2O6"
access_token_secret = "5y0pofXx8zwSNJwCWaPFRK04KYli2E3MAsP29lpsB2tZO"

# Authenticate to Twitter API
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)

# Create API object
api = tweepy.API(auth,wait_on_rate_limit=True)

# Read the CSV file
with open("1000_1500.csv", "r") as f:
    reader = csv.reader(f)
    # Skip the header row
    next(reader)
    # Loop through the rows of the CSV file
    
    for row in reader:
        latitude = row[0]
        longitude = row[1]
        # Search for tweets by location using the Cursor object
        try:
            tweets = tweepy.Cursor(api.search_tweets, q="", geocode=f"{latitude},{longitude},1km", count=1000).items()
        except tweepy.RateLimitError:
            print("Rate limit exceeded. Waiting for 15 minutes...")
            time.sleep(900)
            continue
        
        with open("tweetsspaces2.csv", "w", newline="") as f:
            writer = csv.writer(f)
            
            # Write the header row
            writer.writerow(["Latitude", "Longitude","Date", "User", "Text"])
            
            for tweet in tweets:
                if tweet.coordinates is not None:
                    # Write the tweet text, user name, latitude, and longitude to the CSV file
                    writer.writerow([tweet.coordinates["coordinates"][1], tweet.coordinates["coordinates"][0],tweet.created_at, tweet.user.screen_name, tweet.text])
                        
    time.sleep(1)
        

Rate limit reached. Sleeping for: 774
Rate limit reached. Sleeping for: 766


In [ ]:
import csv
import re

# The input CSV file
input_file = "tweets.csv"

# The output CSV file
output_file = "Istanbultweets.csv"

# Regular expression pattern to match URLs
url_pattern = re.compile(r"(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?")

# Read the input CSV file
with open(input_file, "r") as f_input:
    reader = csv.reader(f_input)
    header = next(reader)
    text_column_index = header.index("Text")
    data = [row for row in reader]

# Split the text and URLs into separate columns
rows = []
for row in data:
    text = row[text_column_index]
    urls = re.findall(url_pattern, text)
    at_index = text.find("@")
    if at_index != -1:
        text_before_at = text[:at_index].strip()
        text_after_at = text[at_index:].strip()
    else:
        text_before_at = text
        text_after_at = ""
    for url in urls:
        text_before_at = text_before_at.replace(url[0] + "://" + url[1] + url[2], "")
        text_before_at = re.sub(r'[^\w\s]', '', text_before_at) # remove symbols
        text_before_at = re.sub(r'#\w+', '', text_before_at) # remove hashtags
        rows.append(row[:text_column_index] + [text_before_at, text_after_at] + [url[0] + "://" + url[1] + url[2]] + row[text_column_index+1:])

# Write the output CSV file
with open(output_file, "w", newline="") as f_output:
    writer = csv.writer(f_output)
    writer.writerow(header + ["Text after @", "URL"])
    writer.writerows(rows)


In [27]:
pip install googletrans

     |████████████████████████████████| 55 kB 3.8 MB/s eta 0:00:011
     |████████████████████████████████| 42 kB 2.7 MB/s  eta 0:00:01
     |████████████████████████████████| 58 kB 9.5 MB/s  eta 0:00:01
     |████████████████████████████████| 133 kB 6.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 15.3 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 3.0 MB/s eta 0:00:01
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15735 sha256=484baf5b4714f867a573484c20a5425a7b811e30cc6a822e83417606e6ae97f6
  Stored in directory: /Users/mansikothari/Library/Caches/pip/wheels/27/f3/32/d4859d40071f07a5df0ab6fdc0076e78a8a786625dde2b4b2f
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: idna
    Found existing installation: 

In [12]:
import csv
import re
from googletrans import Translator

# The input CSV file
input_file = "output.csv"

# The output CSV file
output_file = "English01.csv"

# Read the input CSV file
with open(input_file, "r") as f_input:
    reader = csv.reader(f_input)
    header = next(reader)
    text_column_index = header.index("Text")
    data = [row for row in reader]
    
def translate_text(text):
    try:
        translator = Translator(service_urls=['translate.google.com'])
        translated_text = translator.translate(text, dest='en').text
        return translated_text
    except:
        return text


# Write the output CSV file
with open(output_file, "w", newline="") as f_output:
    writer = csv.writer(f_output)
    writer.writerow(header)
    for row in data[:100]:
        text = row[text_column_index]
        translated_text = translate_text(text)
        row[text_column_index] = translated_text
        writer.writerow(row)